## References

https://www.analyticsvidhya.com/blog/2021/12/fine-tune-bert-model-for-sentiment-analysis-in-google-colab/


https://nyu-mll.github.io/CoLA/

https://colab.research.google.com/github/sudo-ken/compress-decompress-in-Google-Drive/blob/master/Unrar_Unzip_Rar_Zip_in_GDrive.ipynb#scrollTo=O1_kc7HBeslm

https://huggingface.co/docs/datasets/loading_datasets.html

https://huggingface.co/docs/datasets/quickstart.html

https://huggingface.co/docs/datasets/load_hub.html

https://deepai.org/dataset/stanford-sentiment-treebank

https://github.com/huggingface/transformers/issues/400

https://gitanswer.com/transformers-how-to-freeze-bert-model-and-just-train-a-classifier-python-424503196

https://stackoverflow.com/questions/50283844/in-keras-how-to-get-the-layer-name-associated-with-a-model-object-contained-i

https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

https://keras.io/getting_started/faq/#how-can-i-freeze-layers-and-do-finetuning

https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

https://keras.io/api/models/model_training_apis/

https://www.tensorflow.org/guide/keras/train_and_evaluate

https://www.tensorflow.org/api_docs/python/tf/keras/losses

https://swatimeena989.medium.com/bert-text-classification-using-keras-903671e0207d

https://keras.io/guides/transfer_learning/

https://github.com/huggingface/transformers/blob/master/examples/tensorflow/text-classification/run_glue.py

## Imports and installations

In [1]:
!pip install -q transformers
!pip install datasets

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 57.3 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.2 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 306 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 67.8 MB/s 
     |████████████████████████████████| 1.1 MB 65.7 MB/s 
     |████████████████████████████████| 133 kB 68.5 MB/s 
     |████████████████████████████████| 160 kB 73.8 MB/s 
     |████████████████████████████████| 271 kB 72.8 MB/s 
     |████████████████████████████████| 192 kB 72.6 MB/s 


In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
# from transformers import TFBertForSequenceClassification, BertTokenizer
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from tensorflow.keras import models, layers, activations

## COLA dataset

Data Format

Each line in the .tsv files consists of 4 tab-separated columns.

Column 1:	the code representing the source of the sentence.

Column 2:	the acceptability judgment label (0=unacceptable, 1=acceptable).

Column 3:	the acceptability judgment as originally notated by the author.

Column 4:	the sentence.

In [15]:
cola = load_dataset('glue', 'cola')

Downloading:   0%|          | 0.00/377k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
print(cola)
print(cola["train"])

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})
Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 8551
})


In [17]:
print(cola["train"][0])

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'label': 1, 'idx': 0}


## STSB dataset

Data Format:

Column 1:	sentence1

Column 2:	sentence2

Column 3:	label

Column 4:	idx

In [3]:
stsb = load_dataset('glue', 'stsb')

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
print(stsb)
print(stsb["train"])

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})
Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 5749
})


In [5]:
print(stsb["train"][9])

{'sentence1': 'A person is throwing a cat on to the ceiling.', 'sentence2': 'A person throws a cat on the ceiling.', 'label': 5.0, 'idx': 9}


## SST-2 dataset


The Stanford Sentiment Treebank SST-2 dataset contains 215,154 phrases with fine-grained sentiment labels in the parse trees of 11,855 sentences from movie reviews. Models performances are evaluated either based on a fine-grained (5-way) or binary classification model based on accuracy. 

Data Format

Column 1:	idx

Column 2:	label

Column 3:	sentence

In [ ]:
sst2 = load_dataset('glue', 'sst2')

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(sst2)
print(sst2["train"])

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})


In [ ]:
print(sst2["train"][9])

{'sentence': "are more deeply thought through than in most ` right-thinking ' films ", 'label': 1, 'idx': 9}


## Custom Metrics for Keras

In [6]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Prepare datasets

In [7]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
def encode_cola_sst2(examples):
    return tokenizer(examples['sentence'],truncation=True, padding='max_length')

def encode_stsb(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

In [9]:
def dataset_prepare(dataset, mode="cola", single=False):
    if mode in ["cola", "sst2"]:
        encoder_function = encode_cola_sst2
    elif mode == "stsb":
        encoder_function = encode_stsb
    
    if not single:
        encoded_train = dataset["train"].map(encoder_function, batched=True)
        encoded_train_2 = encoded_train.map(lambda examples: {'label': examples['label']}, batched=True)
        encoded_train_2.set_format(type='tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        features_train = {x: encoded_train_2[x]  for x in ['input_ids', 'token_type_ids', 'attention_mask']}
        tfdataset_train = tf.data.Dataset.from_tensor_slices((features_train, encoded_train_2["label"])).batch(32)

        encoded_test = dataset["validation"].map(encoder_function, batched=True)
        encoded_test_2 = encoded_test.map(lambda examples: {'label': examples['label']}, batched=True)
        encoded_test_2.set_format(type='tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        features_test = {x: encoded_test_2[x]  for x in ['input_ids', 'token_type_ids', 'attention_mask']}
        tfdataset_test = tf.data.Dataset.from_tensor_slices((features_test, encoded_test_2["label"])).batch(32)

        return tfdataset_train, tfdataset_test

    else:
        encoded = dataset.map(encoder_function, batched=True)
        encoded_2 = encoded.map(lambda examples: {'label': examples['label']}, batched=True)
        encoded_2.set_format(type='tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        features = {x: encoded_2[x]  for x in ['input_ids', 'token_type_ids', 'attention_mask']}
        tfdataset = tf.data.Dataset.from_tensor_slices((features, encoded_2["label"])).batch(32)

        return tfdataset


In [18]:
cola_train, cola_test = dataset_prepare(cola, mode="cola")

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [10]:
stsb_train, stsb_test = dataset_prepare(stsb, mode="stsb")

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
sst2_train, sst2_test = dataset_prepare(sst2, mode="sst2")

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Train Model

In [24]:
def train_model_on_a_dataset(train_dataset, epochs=3, lr=3e-5, mode="cola"):
    if mode in ["cola", "sst2"]:
        num_labels = 2
    elif mode == "stsb":
        num_labels = 1

    model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=num_labels)
    model.layers[0].trainable = False
    model.layers[1].trainable = False

    opt = tf.keras.optimizers.Adam(learning_rate=lr)

    if mode in ["cola", "sst2"]:
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE, from_logits=True)
        model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy', f1, precision, recall])
    elif mode == "stsb":
        loss_fn = tf.keras.losses.MeanSquaredError(name='mean_squared_error')
        # model_changed = models.Sequential()
        # model_changed.add(models)
        # model_changed.add(layers.Activation(activations.sigmoid))
        # model = model_changed 
        model.compile(optimizer=opt, loss=loss_fn, metrics=[])
        # model.compile(optimizer=opt, loss=loss_fn, metrics=[tf.keras.metrics.RootMeanSquaredError()])

    model.fit(train_dataset, epochs=epochs)

    return model

In [ ]:
cola_model = train_model_on_a_dataset(cola_train, mode="cola")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
268/268 [==============================] - 387s 1s/step - loss: 0.6203 - accuracy: 0.6785 - f1: 0.6622 - precision: 0.7049 - recall: 0.7103
Epoch 2/3
268/268 [==============================] - 386s 1s/step - loss: 0.6095 - accuracy: 0.7045 - f1: 0.7381 - precision: 0.7011 - recall: 0.8132
Epoch 3/3
268/268 [==============================] - 386s 1s/step - loss: 0.6080 - accuracy: 0.7041 - f1: 0.7401 - precision: 0.7038 - recall: 0.8161


In [26]:
stsb_model = train_model_on_a_dataset(stsb_train, mode="stsb", epochs=8)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/8
180/180 [==============================] - 278s 1s/step - loss: 3.7732
Epoch 2/8
180/180 [==============================] - 268s 1s/step - loss: 1.6321
Epoch 3/8
180/180 [==============================] - 268s 1s/step - loss: 1.6141
Epoch 4/8
180/180 [==============================] - 263s 1s/step - loss: 1.6169
Epoch 5/8
180/180 [==============================] - 266s 1s/step - loss: 1.6043
Epoch 6/8
180/180 [==============================] - 262s 1s/step - loss: 1.5876
Epoch 7/8
180/180 [==============================] - 267s 1s/step - loss: 1.5803
Epoch 8/8
180/180 [==============================] - 264s 1s/step - loss: 1.5821


In [ ]:
sst2_model = train_model_on_a_dataset(sst2_train, epochs=1, mode="sst2")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2105/2105 [==============================] - 3041s 1s/step - loss: 0.6673 - accuracy: 0.5896 - f1: 0.6959 - precision: 0.5566 - recall: 0.9494


## Test Model

In [ ]:
results_cola = cola_model.evaluate(cola_test)

33/33 [==============================] - 42s 1s/step - loss: 0.6186 - accuracy: 0.6913 - f1: 0.8151 - precision: 0.6931 - recall: 1.0000


In [27]:
results_stsb = stsb_model.evaluate(stsb_test)

47/47 [==============================] - 60s 1s/step - loss: 1.4525


In [ ]:
results_sst2 = sst2_model.evaluate(sst2_test)

28/28 [==============================] - 35s 1s/step - loss: 0.6320 - accuracy: 0.6720 - f1: 0.6524 - precision: 0.4967 - recall: 0.9502


##  Test on custom input

In [ ]:
def test_on_raw_sample(model, tokenizer=tokenizer, sentence=None):
    
    predict_input = tokenizer.encode(sentence, truncation=True, padding=True, return_tensors="tf")

    tf_output = model.predict(predict_input)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1)
    labels = ['Negative','Positive'] #(0:negative, 1:positive)
    label = tf.argmax(tf_prediction, axis=1)
    label = label.numpy()
    return labels[label[0]]

In [ ]:
sentence = 'Mary listens to the Grateful Dead, she gets depressed.'
test_on_raw_sample(cola_model, tokenizer=tokenizer, sentence=sentence)

'Positive'

In [ ]:
sentence = "are more deeply thought through than in most ` right-thinking ' films "
test_on_raw_sample(sst2_model, tokenizer=tokenizer, sentence=sentence)

'Positive'

In [28]:
predictions_stsb = stsb_model.predict(stsb_test)

In [36]:
def test_on_dataset_sample(model, results, index, dataset, mode="cola"):
    predicted_value = results[0][index]
    real_value = dataset[index]["label"]
    if mode in ["cola", "sst2"]:
        tf_prediction = tf.nn.softmax(predicted_value, axis=1)
        labels = ['Negative','Positive'] #(0:negative, 1:positive)
        label = tf.argmax(tf_prediction, axis=1)
        predicted_value_transformed = label

    elif mode == "stsb":
        predicted_value_transformed = predicted_value[0]

    print(f"Predicted value: {predicted_value_transformed}, Real value is: {real_value}")
    return predicted_value_transformed

In [37]:
test_on_dataset_sample(model=stsb_model, results=predictions_stsb, index=3, dataset=stsb["validation"], mode="stsb");

Predicted value: 2.5230534076690674, Real value is: 2.4000000953674316
